In [1]:

import warnings
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from requests import get
import seaborn as sns
# map functions
import os 
import folium
import rasterio as rio
import earthpy as et
from folium import plugins
from rasterio.warp import calculate_default_transform, reproject, Resampling
from IPython.display import IFrame

import folium
import folium.plugins as plugins

# to ignore the warnings and make the notebook more presentable
warnings.filterwarnings('ignore') 

import pandas as pd
import plotly.io as pio
import plotly.offline as py
import colorlover as cl

from plotly import graph_objects as go
from IPython.display import IFrame

# geometry
from shapely.geometry import Point, LineString
from shapely.geometry import Polygon
from shapely.geometry import Point

from pandas.io.json import json_normalize
pio.renderers.default = 'iframe'
from scipy import stats

import plotly.io as pio
import plotly.express as px
%matplotlib inline

In [2]:
high_risk = pd.read_csv('data/high_risk.csv')

In [3]:
high_risk

,Unnamed: 0,violations,sums_poisoning,violation_type,frequency_poisoning,sums_others,frequency_others,ratio
0,2,"Clean, maintain equipment (32)",374.0,minor,9.873284,45368.0,9.494352,1.039911
1,5,Hot/cold storage temp. (3),231.0,critical,6.098205,16837.0,3.523550,1.730699
2,8,Labelled containers (30),136.0,minor,3.590285,14989.0,3.136811,1.144565
3,9,"Pest control, associated areas (18)",105.0,severe,2.771911,12769.0,2.672222,1.037306
4,10,Sterile utensils (31),104.0,minor,2.745512,9059.0,1.895815,1.448196
5,12,Protection from contamination (16),75.0,severe,1.979937,5388.0,1.127569,1.755933
6,13,Hot/cold storage facilities (2),67.0,critical,1.768743,5597.0,1.171308,1.510059
7,15,"Ice handling, hairnets, clothes (42)",54.0,minor,1.425554,6084.0,1.273224,1.119641
8,16,Correct minor violations (29),54.0,severe,1.425554,6054.0,1.266946,1.125190
9,18,Proper hand washing (6),49.0,critical,1.293559,1850.0,0.387157,3.341171


In [4]:
high_risk= high_risk.replace('critical', 'priority violations (P)')
high_risk= high_risk.replace('severe', 'priority foundation (FP)')
high_risk= high_risk.replace('minor', 'core violation (C)')

In [5]:

fig = px.scatter(data_frame = high_risk, x="violations", y="frequency_poisoning",
         size="ratio", color="violation_type",
                  size_max=60)
fig.show()

In [136]:
minor = high_risk.loc[high_risk['violation_type'] == 'core violation (C)']
severe = high_risk.loc[high_risk['violation_type'] == 'priority foundation (FP)']
critical = high_risk.loc[high_risk['violation_type'] == 'priority violations (P)']

def get_vio_no(df):
    vio_no_list = []
    for i in range(len(df)):
        vio_no = df.violations.values[i]
        vio_no = vio_no[-4:]
        vio_no_list.append(vio_no)
    
    return vio_no_list
    
minor['violation_no'] = get_vio_no(minor)
severe['violation_no'] = get_vio_no(severe)
critical['violation_no'] = get_vio_no(critical)


In [153]:
# plot for violations

fig = go.Figure()


# set up the first trace
fig.add_trace(go.Scatter(
    x = minor.violation_no.tolist(), 
    y = minor.frequency_poisoning.tolist(),
    mode = 'markers',
    name = 'Core violation (C)',
    text = minor.violations.tolist(),
    hovertemplate =
    'Violation: %{text}<br>'+
    'Frequency poisoning: %{y}<br>'+
    'Ratio: %{marker.size}<br>',
    marker = dict(
        size = minor.ratio.tolist(),
        sizemode ='area',
        sizeref = 2. * max(minor.ratio.tolist()) / (60 ** 2),
        sizemin = 4,
        color = 'gold'
        )
    )
)

fig.add_trace(go.Scatter(
    x = severe.violation_no.tolist(), 
    y = severe.frequency_poisoning.tolist(),
    mode = 'markers',
    name = 'Priority foundation (FP)',
    text = severe.violations.tolist(),
    hovertemplate =
    'Violation: %{text}<br>'+
    'Frequency poisoning: %{y}<br>'+
    'Ratio: %{marker.size}<br>',
    marker = dict(
        color = 'darkorange',
        size = severe.ratio.tolist(),
        sizemode ='area',
        sizeref = 2. * max(severe.ratio.tolist()) / (60 ** 2),
        sizemin = 4,
        )
    )
)

fig.add_trace(go.Scatter(
    x = critical.violation_no.tolist(), 
    y = critical.frequency_poisoning.tolist(),
    mode = 'markers',
    name = 'Priority violations (P)',
    text = critical.violations.tolist(),
    hovertemplate =
    'Violation: %{text}<br>'+
    'Frequency poisoning: %{y}<br>'+
    'Ratio: %{marker.size}<br>',
    marker = dict(
        color = 'maroon',
        size = critical.ratio.tolist(),
        sizemode ='area',
        sizeref = 2. * max(critical.ratio.tolist()) / (60 ** 2),
        sizemin = 4,
        ),

    )
)

fig.layout.update(
    autosize = False,
    width = 900,
    height = 500,
    template = "plotly_white",
    #paper_bgcolor='#333'
)

fig.layout.update(
    title = go.layout.Title(
        text = 'Frequency of violations encountered during food poisoning inspections',
        y = 0.9,
        x = 0.5,
        xanchor = 'center',
        yanchor = 'top',
        font = dict(size = 14)
    ),
    showlegend = False,
    yaxis = dict(
        title = 'Violation frequency',
        titlefont_size = 14
    ),
    xaxis = dict(
        title = 'Violations',
        titlefont_size = 14
    )
)

fig.show()

In [71]:
fig = px.scatter(data_frame = high_risk, x="violations", y="frequency_others",
         size="ratio", color="violation_type",
                  size_max=60)
fig.show()